In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
%cd /content/drive/MyDrive/DS/Time_series_project_3

/content/drive/MyDrive/DS/Time_series_project_3


In [3]:
! ls

 data
 ts_forecast
'Генерация_и_отбор_признаков .ipynb'
 Дообучение_и_калибровка_с_разладками.ipynb
'Описание структуры проекта .gdoc'
 Разладки.ipynb


In [ ]:
! pip install holidays
! pip install tsfresh
! pip install fcbf
! pip install skfeature-chappers
! pip install rapidfuzz
! pip install pmdarima

In [1]:
from tqdm import tqdm
import warnings

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.cm import ScalarMappable

from scipy import stats
import statsmodels.api as sm
from itertools import product
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.svm import SVR

import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs
from copy import deepcopy

from statsmodels.tsa.holtwinters import ExponentialSmoothing

from dateutil.relativedelta import relativedelta
import datetime
import holidays

from typing import Union


from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import ComprehensiveFCParameters, EfficientFCParameters
from tsfresh.feature_extraction import settings

In [2]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm

In [3]:
from sklearn.linear_model import Lasso, Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from fcbf import fcbf
from rapidfuzz.distance import Indel

In [4]:
from sklearn.metrics import mean_absolute_error
from sklearn.svm import SVC
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima.arima import auto_arima

In [5]:
import statistics

In [6]:
# params
pd.set_option('max_colwidth', None)
warnings.filterwarnings("ignore", category=DeprecationWarning)
%matplotlib inline

In [7]:
%config InlineBackend.figure_format = 'retina'
plt.rcParams["figure.figsize"] = (15, 8)
# plt.style.use('bmh')
sns.set(style='darkgrid')

In [10]:
# our custom functions
from ts_forecast.feature_engineering import get_features, impute_nans
from ts_forecast.feature_selection import *

# Загрузка подготовленных данных

In [11]:
data = pd.read_csv('data/ts_project_data_full.csv', index_col='Date')
data.index = pd.to_datetime(data.index, format='%Y-%m-%d') # utc=True
data.head()

,Income,Outcome,Balance,Nalog,Moex,Brent,Libor,Rvi,Covid_cases,Covid_deaths,Key_rate,Inflation,Dollar,Euro,Gold
Date,,,,,,,,,,,,,,,
2017-01-09,1.343028,1.487865,-0.155904,1,2211.25,54.94,0.69300,25.59,0,0,10.0,5.0,59.8495,63.0176,2264.82
2017-01-10,1.068610,1.194182,-0.125572,0,2237.49,53.64,0.69333,23.83,0,0,10.0,5.0,59.9672,63.5227,2280.67
2017-01-11,0.944429,0.936663,0.007767,0,2218.61,55.10,0.69278,25.20,0,0,10.0,5.0,60.1430,63.3733,2297.00
2017-01-12,1.672202,0.875379,0.800391,0,2212.00,56.01,0.69278,23.64,0,0,10.0,5.0,59.3784,63.1847,2308.20
2017-01-13,0.955924,0.975645,-0.019721,1,2195.19,55.45,0.69278,24.79,0,0,10.0,5.0,59.3602,63.1565,2283.58


In [12]:
print(f"Min date: {data.index.min()}")
print(f"Max date: {data.index.max()}")
print(f"Number of days in dataset: {data.shape[0]}")

Min date: 2017-01-09 00:00:00
Max date: 2021-03-31 00:00:00
Number of days in dataset: 1543


In [13]:
data.drop(columns = ['Income', 'Outcome'], inplace=True)

In [14]:
print("Факторы:")

for col in data.columns[1:-1]:
  print(col)

Факторы:
Nalog
Moex
Brent
Libor
Rvi
Covid_cases
Covid_deaths
Key_rate
Inflation
Dollar
Euro


Заменяем `nan` предыдущими значениями

In [15]:
cols = ['Balance', 'Nalog', 'Moex', 'Brent', 'Libor', 'Rvi', 'Covid_cases',
       'Covid_deaths', 'Key_rate', 'Inflation', 'Dollar', 'Euro']

In [16]:
dataset = data[cols].copy()

In [17]:
impute_nans(dataset)

# Генерация признаков

In [18]:
ru_holidays = holidays.RU()
df = get_features(dataset, ru_holidays)
df.shape

(1423, 76)

In [19]:
df.columns

Index(['Balance', 'Nalog', 'Moex', 'Brent', 'Libor', 'Rvi', 'Covid_cases',
       'Covid_deaths', 'Key_rate', 'Inflation', 'Dollar', 'Euro',
       'salary_payments', 'weekends', 'first_day_of_month', 'weekday_0',
       'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5',
       'weekday_6', 'month_1', 'month_2', 'month_3', 'month_4', 'month_5',
       'month_6', 'month_7', 'month_8', 'month_9', 'month_10', 'month_11',
       'month_12', 'quarter_1', 'quarter_2', 'quarter_3', 'quarter_4',
       'balance_rolling_mean_weekday', 'balance_rolling_max_weekday',
       'balance_rolling_min_weekday', 'balance_rolling_median_weekday',
       'balance_rolling_std_weekday', 'balance_lag_day_1', 'balance_lag_day_2',
       'balance_lag_day_3', 'balance_lag_day_4', 'balance_lag_day_5',
       'balance_lag_day_6', 'balance_lag_day_7', 'balance_lag_week_2',
       'balance_lag_week_3', 'balance_lag_week_4', 'balance_lag_month_1',
       'balance_lag_month_2', 'balance_lag_month_3', 'me

# Отбор признаков


Модуль отбора признаков содержит в себе:
* **L1-регуляризацию** - встроенный метод, дающий нерелевантным признакам нулевые веса
* **Fast Correlation-Based Filter (FCBF)** - фильтрационный метод, исследующий нелинейнцю зависимость и работающий быстрее и эффективнее, чем ReliefF и CFS(Correlation-based feature selection), и поэтому чаще использующийся для входных данных высокой размерности. Он учитывает релевантность и избыточность, в рамках которого сначала для всех признаков вычисляются Symmetrical Uncertainty (взаимная информация между X и Y I(X, Y), деленная на сумму их энтропий), затем признаки сортируются по этому критерию, а потом удаляются избыточные.
* ****

In [20]:
X = df.drop(columns='Balance')
y = df['Balance']

In [21]:
%%time
res_1, res2, df_res = stable_features(X, y)

/usr/local/lib/python3.10/dist-packages/fcbf/fcbf.py:53: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  return -1 * prod.sum(axis=0, level=y.name).mul(prior(y)).sum(axis=0)
/usr/local/lib/python3.10/dist-packages/fcbf/fcbf.py:53: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  return -1 * prod.sum(axis=0, level=y.name).mul(prior(y)).sum(axis=0)
/usr/local/lib/python3.10/dist-packages/fcbf/fcbf.py:53: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  return -1 * prod.sum(axis=0, level=y.name).mul(prior(y)).sum(axis=0

CPU times: user 27min 56s, sys: 40.7 s, total: 28min 37s
Wall time: 17min 35s


Наиболее оптимальная конфигурация с точки зрения комбинации стабильности, мае и метрики, сконструированной согласно бизнес-процессу

In [22]:
df_res[df_res.Metrics_range==df_res.Metrics_range.max()]

,Features,Model,Stability,MAE,Business,Business_norm,MAE_norm,Metrics_range
14,"[balance_lag_day_7, balance_lag_week_4, Moex, balance_lag_day_6]",Xgboost,0.258333,0.165289,-1.913642,0.959432,0.04075,0.261355


In [24]:
df_res.sort_values(by='Metrics_range', ascending=False)

,Features,Model,Stability,MAE,Business,Business_norm,MAE_norm,Metrics_range
14,"[balance_lag_day_7, balance_lag_week_4, Moex, balance_lag_day_6]",Xgboost,0.258333,0.165289,-1.913642,0.959432,0.040750,0.261355
0,"[balance_rolling_mean_weekday, balance_lag_day_1, Nalog]",LogReg,0.258333,0.162517,-2.507023,0.761899,0.000000,0.251686
12,"[balance_lag_day_7, balance_lag_week_4, Moex, balance_lag_day_6]",LogReg,0.258333,0.171545,-1.791775,1.000000,0.132732,0.249406
1,"[balance_rolling_mean_weekday, balance_lag_day_1, Nalog]",SVM,0.258333,0.165882,-2.642082,0.716938,0.049471,0.239694
3,"[balance_lag_day_7, balance_lag_week_4, mean_30]",LogReg,0.258333,0.169979,-2.299095,0.831116,0.109714,0.239169
9,"[balance_lag_day_7, balance_lag_week_4, mean_3]",LogReg,0.258333,0.166790,-2.689806,0.701051,0.062814,0.236146
16,"[balance_rolling_mean_weekday, balance_rolling_std_weekday, std_3, max_3, min_3, median_14]",SVM,0.083333,0.165961,-2.250596,0.847262,0.050639,0.235776
15,"[balance_rolling_mean_weekday, balance_rolling_std_weekday, std_3, max_3, min_3, median_14]",LogReg,0.083333,0.166865,-2.238193,0.851390,0.063923,0.233906
27,"[weekday_0, balance_rolling_mean_weekday, balance_rolling_median_weekday, median_7, std_14, median_30]",LogReg,0.083333,0.165078,-2.477095,0.771861,0.037650,0.231658
17,"[balance_rolling_mean_weekday, balance_rolling_std_weekday, std_3, max_3, min_3, median_14]",Xgboost,0.083333,0.167119,-2.475812,0.772289,0.067650,0.226693
